In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import time
import pandas as pd

In [2]:
def scrape_ebay_data(keyword, start_page, end_page):
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    service = Service('chromedriver')
    driver = webdriver.Chrome(service=service, options=options)

    data_list = []

    for page_number in range(start_page, end_page + 1):
        link = f'https://www.ebay.com/sch/i.html?_from=R40&_nkw={keyword}&_sacat=0&_pgn={page_number}'
        driver.get(link)
        time.sleep(5)

        content = driver.page_source
        data = BeautifulSoup(content, 'html.parser')
        items = data.find_all('div', class_='s-item__wrapper clearfix')

        print(f"Scraping page {page_number}, found {len(items)} items...")

        for area in items:
            item_data = {}

            nama = area.find('span', role='heading')
            item_data['name'] = nama.text.strip() if nama else 'N/A'

            con = area.find('span', class_='SECONDARY_INFO')
            item_data['condition'] = con.text.strip() if con else 'N/A'

            loc = area.find('span', class_='s-item__location s-item__itemLocation')
            item_data['location'] = loc.text.strip() if loc else 'N/A'

            price = area.find('span', class_='s-item__price')
            item_data['price'] = price.text.strip() if price else 'N/A'

            url = area.a['href']
            item_data['link'] = url.strip() if url else 'N/A'

            data_list.append(item_data)

        print(f"Scraping page {page_number}...")

    driver.quit()
    return data_list

if __name__ == "__main__":
    keyword = input("Enter keywords: ")
    start_page = int(input("Enter the start page number: "))
    end_page = int(input("Enter the final page number: "))

    scraped_data = scrape_ebay_data(keyword, start_page, end_page)

    if scraped_data:
        df = pd.DataFrame(scraped_data)
        df.to_csv(f"{keyword}_ebay_data.csv", index=False)
        print("Data successfully saved to CSV file.")
    else:
        print("No data found.")

print('Scrape Done')

Enter keywords: tv
Enter the start page number: 1
Enter the final page number: 6
Scraping page 1, found 61 items...
Scraping page 1...
Scraping page 2, found 61 items...
Scraping page 2...
Scraping page 3, found 61 items...
Scraping page 3...
Scraping page 4, found 61 items...
Scraping page 4...
Scraping page 5, found 0 items...
Scraping page 5...
Scraping page 6, found 0 items...
Scraping page 6...
Data successfully saved to CSV file.
Scrape Done


In [4]:
df = pd.read_csv("tv_ebay_data.csv")
df

,name,condition,location,price,link
0,Shop on eBay,Brand New,NaN,$20.00,https://ebay.com/itm/123456?hash=item28caef0a3...
1,"Onyx Smart TV 4K LED TV 43"""" inch 108 cm",Brand New,from Germany,"IDR3,341,060.61",https://www.ebay.com/itm/134672269452?hash=ite...
2,"T4tec 24"" Inch HD Ready LED TV and Freeview Play",Brand New,from United Kingdom,"IDR1,657,049.24",https://www.ebay.com/itm/134613866900?hash=ite...
3,"16” LED HD TV 12V 240V DVB-T2, FM Tuners Freev...",Brand New,from United Kingdom,"IDR2,658,296.88",https://www.ebay.com/itm/165666023171?hash=ite...
4,"JVC LT-32CA120 Android TV 32"" Smart HD Ready H...",Excellent - Refurbished,from United Kingdom,"IDR2,300,374.24",https://www.ebay.com/itm/125710387198?epid=210...
...,...,...,...,...,...
239,"LG OLED55G36LA 55"" OLED TV / BLEMISHES (1321)",Good - Refurbished,from United Kingdom,"IDR25,323,611.36",https://www.ebay.com/itm/256141509913?epid=130...
240,"COBY CX-TV1 Compact Portable 5"" Black & White...",Brand New,from United States,"IDR136,363.64",https://www.ebay.com/itm/385838358948?epid=482...
241,"Tyler 16"" Portable TV LCD Monitor Battery Powe...",Brand New,from United States,"IDR2,393,787.88",https://www.ebay.com/itm/394105632525?hash=ite...
242,Emby TV 1 month Access 4K With the 100% satisf...,Brand New,from United Kingdom,"IDR194,752.03",https://www.ebay.com/itm/125886735277?hash=ite...
